# BME 426 - SQ21

MRI Modeling of Brain Physiology

Prof. Molly Bright (molly.bright@northwestern.edu)

## Analysis Assignment #1
**Instructions:** In this assignment, you will quantify relaxation time constants in simulated and real data.

You will need the following: **AA1_data.mat** This file contains vectors for simulated $T_1$ and $T_2$ signal decays, as well as a multi-echo gradient echo (GRE) dataset. Make sure that the data files are in the directory you are working from, or that you know the full path to the directory and edit the code below accordingly.


As always, we start the code by importing different packages:

***import scipy***

***from scipy.io import loadmat***

***import numpy as np***

***import matplotlib.pyplot as plt***

In this assignment, you may also need these functions:
- **loadmat()** - reads in a .mat data file; make sure you are in the same directory as the .mat file, or else include the full path to the file
- **np.exp()** - returns the exponential of the argument; e^x
- **np.log()** - returns the natural logarithm of the argument
- **np.vstack().T** - stacks the arrays to create matrix and transposes
- **np.linalg.lstsq(X,Y,Rcond=None)[0]** - this solves a system of linear equations: Y = X$\beta$, where Y is the data, X is the model, and returns $\beta$ ("Beta weights"), the solution that minimizes the sum squared error of the fit (minimizes $\|Y-X\beta\|$). Note, the "[0]" results in only $\beta$ being returned, but the function can also return the residuals, rank, etc. if indicated.
- **np.matmul(J,K)** - performs matrix multiplication J * K
- **np.divide(J,K)** - performs element-wise division J/K
- **plt.imshow(x, cmap = )** - this displays a 2D matrix as an image, and lets you change the scaling of the image brightness and color
- **fig, ax = matplotlib.pyplot.subplot()** - this lets you divide up one figure into multiple subplots and use the same axis
- **plt.title()** - this lets you add a title (input as a string) to a figure or subplot
- **plt.xlabel()** - this lets you add a label (input as a string) to the x-axis of a figure or subplot
- **plt.ylabel()** - as above, for y-axis

There may be multiple ways to achieve what the assignment asks you to do, but the above functions could be helpful. If you aren't familiar with the functions above, remember to try looking at help documentation. When there is code already present in the code boxes below, it is recommended that you leave this code and add to it: it is there to be helpful!

Make sure to try running your code at each intermediate step.

For plain text answers, please supply your response in a Text box, not a Code box, where prompts are given.

The **EXTRA** section is optional, but everyone is encouraged to try for better understanding of the concepts at hand!


In [ ]:
# Leave this section of code - this sets up the necessary functions for completing the assignment.

import scipy
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls drive/MyDrive/BME426

AA1_data.mat                      BME495_AnalysisAssignment1_Solutions.ipynb
BME495_AnalysisAssignment1.ipynb


In [ ]:
# Leave this section of code - this sets up your data files.

data = loadmat('/content/drive/MyDrive/BME426/AA1_data.mat')
data1 = data['data1']
data1=data1[0]
data2 = data['data2']
data2=data2[0]
mGRE = data['mGRE']
TEs=data['TEs']
TEs=TEs[0]
time1=data['time1']
time1=time1[0]
time2=data['time2']
time2=time2[0]

### Fitting for T<sub>2</sub>
$T_2$ represents relaxation of the net magnetization vector in the transverse plane, primarily related to spin-spin interactions. Let's look at simulated data representing a $T_2$ decay curve, showing the signal measured at different echo times, and practice fitting for the $T_2$ relaxation time constant using a linear system of equations.

In a $T_2$ imaging experiment, we excite the spins at time _t=0_ and then induce a "spin echo" at time _t_, when we measure a signal intensity _S(t)_.

We know that $T_2$ decay follows the form:

<center>$S(t) = S_0e^{-\frac{t}{T_2}}$</center>

where $T_2$ is the transverse relaxation time constant in units of _ms<sup>-1</sup>_. Although we cannot instantaneously generate a spin echo immediately after the excitation pulse (_t=0_), $S_0$ is an estimate of what signal we would measure if we could. In other words, $S_0$ is the initial signal we can excite before any spin-spin interactions cause dephasing and subsequent loss of that signal.

In the simulated data, the vector of the signal $S(t)$ is named **data2** (arbitrary units) and the vector of echo times $t$ is **time2** (in milliseconds).

First, plot the signal versus echo time, using dots to represent the datapoints rather than a continuous line. It should look like an exponential decay. Make sure to label your figure and axes!

In [ ]:
plt.figure()
plt.plot(time2,data2,'.'); #Plotting function to get you started, make sure to add labels etc.



Next, we want to somehow linearize these data to make fitting easier. This should be familiar from class! Because the signal follows an exponential decay, we can take the log of both sides of the above equation. Then the *log of the signal* can be written as a linear function of time.
<center>
$S(t)=S_{0}e^{-\frac{t}{T_2}}$

$\ln\left(S(t)\right) = \ln(S_0e^{-\frac{t}{T_2}})$

$\ln\left(S(t)\right) = \ln(S_0) + \ln(e^{-\frac{t}{T_2}})$

$\ln\left(S(t)\right) = \ln(S_0) - \frac{1}{T_2}t$ 
</center>


Plot the natural log of $S_0$ as a function of time to observe the linear relationship, again using dots as data marker points.

We talked in class about writing systems of linear equations in matrix form. We can do this here! 

We want to fit a line to the linearized data:
<center>
$y = b + mx$
</center>

where $b$ is the intercept and $m$ is the slope. 

Given the equation determined above:
<center>
$\ln\left(S(t)\right) = \ln(S_0) - \frac{1}{T_2}t$
</center>

it should be clear how **m** and **b** relate to our original unknown parameters and variables in this assignment.

To solve this equation, we can rewrite this linear relationship in matrix form
<center>
$Y = X \beta$
</center>

where

**Y** is the log of the signal _S(t)_

**X** contains two columns, the first being all ones to model the intercept and the second being the time vector to model the slope.

$\beta$ will be the fitted parameters for the intercept and slope (e.g., $b$ and $m$ in the earlier equation).

<center>
$
\begin{bmatrix} Y_{t1}\\ Y_{t2}\\ .\\ .\\ Y_{tn} \end{bmatrix} 
    =
    \begin{bmatrix} 1 & t_1\\ 1 & t_2 \\ . & . \\ . & . \\ 1 & t_n \end{bmatrix}
    \begin{bmatrix} \beta_{1} \\ \beta_{2} \end{bmatrix}
$
</center>

To solve for $\beta$ in this system of linear equations, we will use the numpy function **linalg.lstsq** to return the least-squares solution to the linear matrix equation. In other words, this function will determine $\beta$ such that it minimizes the euclidean norm $\|Y-X\beta\|$. This solution minimizes the sum of the squared errors of the fit.

Define **X** and **Y** and use **np.linalg.lstsq** to determine $\beta$.


In [ ]:
from scipy import linalg  # This imports the function you will need.



Print the slope and intercept.

In [ ]:
print('The slope is {0:1.3f}'.format(m))
print('The intercept is {0:1.3f}'.format(b))

Now create the fit results by multiplying **X** and the fitted parameters ($\beta$), and plot this as a red line on top of the linearized data. Add a legend.

Use the fitted Beta weights to determine the T<sub>2</sub> of the simulated data:

What was the T2 value estimated in your fit? **<< Insert Response Here >>**

### Fitting for T1
$T_1$ is the "longitudinal relaxation time" of a tissue. It reflects the return of the net magnetization vector to its initial maximum value aligned with $B_0$, following an excitation into the XY plane. Let's look at a simulation of a $T_1$ relaxation curve and practice fitting for this relaxation time constant using a linear model as described above.

We know that T<sub>1</sub> relaxation follows the form:

<center>
$M_z=M_{0}\left(1-e^{- \frac{t}{T_1}}\right)$
</center>

The vector of the magnetization vector $M_z$ is saved as **data1** and the vector of times $t$ (in milliseconds) is **time1**.

First, plot the exponential relaxation of the data. Use dots as data markers rather than a line, and make sure to always label your figure elements!

Now linearize the data, ploting the log of M as a function of time (also using dots as data markers). This will be similar to what you did above, but not exactly the same! It may be helpful to write out the equation and step through taking the log of both sides...

Following the instructions from above, fit a line to the linearized data. Remember, $y = mx + b$ can be represented as $Y=X\beta$.

Define **X** and **Y** and use **np.linalg.lstsq()[0]** to determine $\beta$.

Write out the fitted intercept and slope.

Now create the fit results by multiplying **X** and the fitted $\beta$ parameters, and plot this as a red line on top of the linearized data. Add a legend.

Use the fitted Beta weights ($\beta$) to determine the T<sub>1</sub> of the simulated data:

What was the T1 value estimated in your fit? **<< Insert Response Here >>**

### Fitting T2* decay of real data
The array **mGRE** contains a single-slice multi-echo gradient echo (GRE) dataset acquired at 7 Tesla. The times (in milliseconds) of when the multiple echoes were acquired are stored as **TEs**.

First, determine the dimensions of the input data and identify how many echo times were acquired.

Display the data in three horizontal subplots of one figure, in grayscale, showing the image at three different echo times. (Use the matplotlib **imshow()** function.) Make sure to label each subplot to indicate what echo time (TE) is being displayed. (Hint, you may want to format the echo time for easier viewing in the subplot title!)

In [ ]:
plt.subplot(131) #Keep this, this sets up your figure to have 1-by-3 subplots, and makes the first subplot active.


plt.subplot(132) #Keep this, this makes the second subplot active.


plt.subplot(133) #Keep this, this makes the third subplot active.




Next, let's extract from the mGRE dataset the timeseries from one voxel that will show how the signal decays over the multiple gradient echoes. We need to select coordinates for a voxel that you identify to be within cortical GM

**NOTE: Use one of the figures from above to estimate the coordinates of a voxel you believe is in cortical gray matter.** On top of the image, plot two red lines, one horizontal and one vertical, that cross at your chosen coordinate. If you are happy with the location of the voxel, proceed to the next step, or adjust your coordinates until you are happy that you are selecting a good voxel.

_Be careful though, the axes are a bit tricky with imshow_

In [ ]:
#Insert a pair of coordinates here. Note, the x-axis is visualized vertically and the y-axis is visualized horizontally when using imshow().
hor = 
vert = 

#Use imshow to plot the image data and add red horizontal/vertical lines


#Compensate for the swapped axes!
x = vert
y = hor

Next, plot the exponential decay of the T<sub>2</sub>* -weighted signal from this voxel across the echo times. Make sure to label all axes!

Next, repeat the procedures used above to linearize the data and fit for T2*. 
Remember,  \( S(t) = S_0e^{-\frac{t}{T2*}} \) 

What was the T2* value estimated in your fit? **<< Insert Response Here >>**

Finally, do the same procedures for a white matter voxel. 

In [ ]:
# as above with different input coordinates

What was the T2* value estimated in your fit? **<< Insert Response Here >>**

How do your estimated values for T2* in these two tissue classes compare with literature values? Remember, this is 7T data! (A few sentences will suffice, and you may well find that your estimates are very different from literature values. This will not impact your grade if you have gone through the steps correctly.) **<< Insert Response Here >>**

### Maximizing tissue contrast ###

We can use our understanding of the relaxation times to consider the contrast in signal intensity between different tissue types. Using simulated signal decays, we can identify what echo time would give us maximal contrast between, say, gray and white matter tissue.
Let's assume T<sub>2</sub> values (in milliseconds) for white matter and gray matter in the human brain at 3T (Wanaspura et al. JMRI 1999):

In [ ]:
T2wm = 74;
T2gm = 110;

Let's also assume a very long TR, such that we do not emphasize T<sub>1</sub>-weighting. What echo time would give you maximal contrast between these two tissue classes in a T<sub>2</sub>-weighted image?

First, create and plot the theoretical signal decays of gray matter (red dots) and white matter (blue dots) in the same figure.

In [ ]:
t = np.linspace(0,300,60) #Create a vector of timepoints (ms) at which to simulate the signals


Then, calculate and plot the difference between the two decay curves as a green line. This is the contrast between the two tissues!

Finally, identify at what echo time the maximum contrast occurs:

The echo time with maximal contrast is: **<< Insert Response Here >>**

Qualitatively, if we collected a T2-weighted scan using the echo time you just calculated, what would we expect to see in the image, compared to collecting the same scan with a different echo time? Why might this be helpful or desirable?

**<< Insert Response Here >>**

Let's say you wanted to collect the data much faster, so you shorten your TR (time between excitations). What would you need to consider to still maximize tissue contrast? (You do not need to do the math, just explain in a few sentences your thought process.

**<< Insert Response Here >>**

Imagine now we have a two different T2-weighted scans, one with very high spatial resolution and one with very low spatial resolution. What might happen to our T1 and T2 estimates for GM and WM in the poor resolution scan, and why?

**<< Insert Response Here >>**

### FEEDBACK ###
To help gauge the analysis assignments for this new course, please tell me how long this assignment took you to complete. This has no influence on your grade!

**<< Insert Response Here >>**

#### EXTRA ####
We discussed in class the Inversion Recovery experiment. Identify the inversion time $TI$ that will "null" the signal for Gray Matter. Assume that the $T_1$ of gray matter is 1322ms, that $TR$ is extremely long, and $TE$ is extremely short. (With these assumptions, several terms in the IR signal equation can be ignored!)

<center>
$S = S_0 \left[1 - 2\left( e^{-\frac{TI}{T_1}} \right) + e^{-\frac{TR}{T_1} } \right]e^{- \frac{TE}{T_2}} $
</center>

In [ ]:
T1gm = 1322;
